In [59]:
# Preston Zheng and Shirley Zhao
# NOTE: Grocery Shop and Plant Shop are IN DEVELOPMENT!

import random
import time

# Dictionary of all available items
# Item Name : Category

item_type = {

    # GARDENING
    "Mango Seeds" : "g",
    "Wheat Seeds" : "g",
    "Apple Seeds" : "g",

    # CONSUMABLES
    "Bread" : "c",
    "Carrot" : "c",
    "Apple" : "c",
    "Meat" : "c",
    "Mushrooms" : "c",
    "Berries" : "c",

    # TOOLS
    "Sword" : "t",
    "Pickaxe" : "t",

    # RESOURCES
    "Wood" : "r",
    "Stick" : "r",
    "Flowers" : "r",
    "Pebble" : "r",
    "Acorn" : "r",
    "Cobblestone" : "r",
    "Coal" : "r",
    "Iron" : "r",
    "Gold" : "r",
    "Diamond" : "r"

}

# Dictionary of all animals
# Available Biome : Animal species and Animal type
animal_list = {

    "Forest" : [
        {"species" : "Deer", "animal_type" : "killable"},
        {"species" : "Dog", "animal_type" : "friendly"},
        {"species" : "Rabbit", "animal_type" : "killable"},
        {"species" : "Cat", "animal_type" : "friendly"}
    ],

    "Meadow" : [
        {"species" : "Sheep", "animal_type" : "killable"},
        {"species" : "Horse", "animal_type" : "friendly"},
        {"species" : "Cow", "animal_type" : "killable"},
    ],

    "Caves" : [
        {"species" : "Goblin", "animal_type": "killable"},
        {"species" : "Rat", "animal_type": "friendly"},
        {"species" : "Giant Spider", "animal_type": "killable"},
    ]

}

# Player Class

In [61]:
# Initialize Player with name and stats
class Player:
  def __init__(self, username):
    self.username = username

    # Starting inventory
    self.inventory = {"Mango Seeds" : 1,
                      "Wheat Seeds" : 1,
                      "Apple Seeds" : 1,
                      "Meat" : 2,
                      "Bread" : 2,
                      "Iron" : 3,
                      "Diamond" : 1
                      }

    # 50 coziness by default, and start with empty garden and pets
    self.coziness = 50
    self.garden = []
    self.pets = []

  # Adds points to coziness meter
  def relax(self):

    # Stops if relaxing will exceed maximum
    if self.coziness + 10 > 100:
      print("You're already cozy enough!")

    # Adds a delay and adds 10 points to coziness
    else:
      print(f"{self.username} is relaxing!")
      time.sleep(3) # https://www.geeksforgeeks.org/sleep-in-python/
      self.coziness += 10
      print(f"Coziness is at {self.coziness}.")

  # Prints out inventory
  def show_inventory(self):

    # If empty, print out empty
    if self.inventory == {}:
      print("Your inventory is empty!")

    # Loops through all keys and values in inventory, printing them out
    else:
      print("\nInventory:")
      for item, quantity in self.inventory.items(): # https://www.w3schools.com/python/python_dictionaries_loop.asp
        print(f"{item}: {quantity}")
      print("")

  # Obtain function adds items to inventory. Default amount is 1
  def obtain(self, item, amount=1):

    # Checks if already in inventory to avoid duplicates. If so, adds onto value
    if item in self.inventory:
      self.inventory[item] += amount

    # If not, creates new entry in inventory
    else:
      self.inventory[item] = amount

  # Remove function deletes item from inventory
  def remove(self, item, amount=1):

    # Removes item from inventory by amount
    self.inventory[item] -= amount

    if self.inventory[item] <= 0:
      del self.inventory[item]

  # Create list of consumables from inventory
  def inventory_consumables(self):

    # Initial dict of consumables
    consumables = {}

    # Loop through inventory for items with attribute "c"
    for item, quantity in self.inventory.items():
      if item_type.get(item) == "c":

        # If they do, add entry to consumables
        consumables[item] = quantity

    # Turn into a list of tuples and return it
    consumables_list = list(consumables.items()) # https://www.geeksforgeeks.org/convert-a-dictionary-to-a-list-in-python/
    return consumables_list

  # Consume asks user what to consume to add coziness points
  def consume(self): #### NUMBER INPUTS FOR CHOICES

    # Initialize consumables from last function
    consumables = self.inventory_consumables()

    count = 1

    # Loop through the available consumables and number them.
    if consumables == {}:
      print("No consumables.")
      return

    print("")
    for item, quantity in consumables:
      print(f"{count}. {item} : {quantity}")
      count += 1
    print("")

    try:
      # - 1 because Python begins from 0
      # Takes choice of what to consume
      choice = int(input("\nWhich item would you like to consume?")) - 1

      # If number is in range
      if choice < 0 or choice >= len(consumables):
        print("Invalid number.")

      # Define item and quantity based off the tuple generated from consumables[choice]
      item, quantity = consumables[choice] # https://note.nkmk.me/en/python-multi-variables-values/

      # If amount in inventory greater than 0, take one away
      if self.inventory[item] > 0:
        self.inventory[item] -= 1

        # If equal to 0, you ran out and entry should be deleted from inventory
        if self.inventory[item] == 0:
          del self.inventory[item]

      # Adds coziness; Caps at 100
      if self.coziness + 25 >= 100:
        self.coziness = 100
      else:
        self.coziness += 25
      print(f"You've consumed 1 {item}.")
      print(f"You've gained 25 coziness! You have {self.coziness} coziness now.")
    except ValueError:
      print("Numbers only!")

  # Add pet to pet list
  def add_pet(self, animal):
    self.pets.append(animal)

  def show_pets(self):
        if self.pets:
            print(f"{self.username}'s Pets:")
            for pet in self.pets:
                print(f"- {pet.name} the {pet.species}")
        else:
            print(f"{self.username} has no pets yet.")

# Animal Class

In [63]:
class Animal:
  def __init__(self, species, animal_type):
    self.species = species
    self.animal_type = animal_type
    self.friendship_level = 0
    self.name = None
    # self.animal_type = animal_type

  # Befriend checks if there is food available to feed
  # If so, choose a food
  def befriend(self):
    feeding_food = player.inventory_consumables()

    if feeding_food:

      # Continues as long as friendship level is below 50
      while self.friendship_level < 50:
        print(f"\nChoose which food you'd like to feed the {self.species}! \nType 0 to stop.")

        count = 1
        print("")
        for item, quantity in feeding_food:
          print(f"{count}. {item} : {quantity}")
          count += 1

        # Define choice as the food you want to choose
        try:
          choice = int(input("\nEnter the number of the item to use.")) - 1
        except ValueError:
          print("Invalid number.")
          continue

        if choice == -1:
            print(f"The {self.species} ran away!")
            return

        # Accesses tuple list, finds the number of the choice and then the first index (item name)
        item = feeding_food[choice][0]

        # If choice is within range, remove item from inventory and add 25 to friendship level.
        if 0 <= choice < len(feeding_food):
          player.remove(item)
          # Update list
          feeding_food = player.inventory_consumables()
          self.friendship_level += 25
          print(f"You fed the {self.species} a {item}! Friendship level: {self.friendship_level}.")
        else:
          print("Input out of range. Try again.")

    else:
      print("You don't have any food to give!")
      return

      if self.friendship_level >= 50:
        pet_name = input(f"Congrats! You've befriended the {self.species}. What would you like to name it? ")
        self.name = pet_name
        print(f"You've named your {self.species} {self.name}!")
        player.add_pet(self)

  # Kills animal and gives you 5 meat if there is a sword in inventory
  def kill(self):
    if "Sword" in player.inventory:
      print(f"You killed the {self.species}!")
      player.obtain("Meat", 5)
    else:
      print("You need a sword to kill this animal.")


# Biome Class

In [65]:
# Biome class: has name, list of obtainable resources, and a rule to unlock
class Biome:

  def __init__(self, name, rule=None):
    self.name = name
    self.unlocked = True
    self.rule = rule
    self.resources = []

  # Encounter function creates new instance of Animal
  # Generate random animal from chosen biome with random attributes
  def animal_encounter(self):
    animal_attributes = random.choice(animal_list[self.name])
    animal = Animal(animal_attributes["species"], animal_attributes["animal_type"])

    # If friendly, you can befriend or do nothing
    if animal.animal_type == "friendly":
      print(f"\nYou discovered a friendly {animal.species}!")
      action = input("What would you like to do? \n1. Befriend \n2. Do Nothing\n")
      while True:
        if action == "1":
          animal.befriend()
          return False
        elif action == "2":
          return False
        else:
          "Invalid action, try again."

    # If killable, you can kill or do nothing
    elif animal.animal_type == "killable":
      print(f"\nYou discovered a killable {animal.species}!")
      action = input("What would you like to do? \n1. Kill \n2. Do Nothing\n")
      while True:
        if action == "1":
          animal.kill()
          return False
        elif action == "2":
          return False
        else:
          "Invalid action, try again."
      player.coziness -= 10

  # Explore function to randomly choose from list of resources
  def explore(self, player):

    # 75% probability to get a resource
    if random.random() > 0.25:

      resource = random.choice(self.resources)
      print(f"Exploring the {self.name}...")
      time.sleep(2)
      print(f"You found {resource}!")
      player.obtain(resource, 1)

    # 25% probability to encounter an animal
    else:
      self.animal_encounter()

  # Checks if rule is fulfilled and sets unlocked status to true
  def unlock(self, player):
    if self.check_rule(player):
      self.unlocked = True
      print(f"{self.name} is now unlocked!")
    else:
      print(f"{self.name} is still locked.")

# Forest subclass
class Forest(Biome):
  def __init__(self):
    super().__init__("Forest")
    self.resources = ["Wood",
                      "Stick",
                      "Acorn",
                      "Apple",
                      "Apple Seeds"
                      ]

  def check_rule(self, player):
    return True

# Meadow subclass - can relax here
class Meadow(Biome):
  def __init__(self):
    super().__init__("Meadow")
    self.resources = ["Wheat Seeds",
                      "Flowers",
                      "Mushrooms",
                      "Berries",
                      "Pebbles",
                      "Mango Seeds"
                      ]

  def check_rule(self, player):
    return True

class Caves(Biome):
  def __init__(self):
    super().__init__("Caves")
    self.resources = ["Cobblestone",
                      "Iron",
                      "Diamond",
                      "Gold",
                      "Coal"
                      ]

  def check_rule(self, player):
    if "Pickaxe" in player.inventory:
        return True
    else:
        print("You need a pickaxe to travel to Caves!")
        return False

# Town subclass - can enter stores and your house
class Town(Biome):
  def __init__(self):
    super().__init__("Town")
    self.places = {
        "Blacksmith": "Tools and metals.",
        "Bakery": "Fresh bread and pastries!",
        "Grocery Store": "Consumables sold here.",
        "Plant Shop": "Gardening seeds!",
        "House": "Your home - where you can relax or tend to your garden"
    }
    print("")

  def check_rule(self, player):
    return True

  # Overrides explore function so now it prints a directory of places to go.
  def explore(self, player):
    print("\nWelcome! You can visit the following places:")

    # Loops through places dictionary to show places and descriptions
    print("")
    for place, description in self.places.items():
      print(f"{place} : {description}")
    choice = input("Where would you like to visit?")

    # Enter house stage, where you can garden or relax
    if choice.lower() == "house":

      while True:
        print("\nYou're in your house! You can garden, relax, show inventory, or leave.")
        choice = input("What would you like to do: ")

        # Enter garden stage, where you can tend to your plants
        if choice.lower() == "garden":
          garden.garden_loop(player)

        # Relax at home.
        if choice.lower() == "relax":
          player.relax()

        # Show inventory
        if choice.lower() == "show inventory":
          player.show_inventory()

        # Leave to go back to town.
        if choice.lower() == "leave":
          return False

        else:
          print("Invalid input")

    # Enter NPC shops
    elif choice.capitalize() in self.places:
      print(f"You have entered {choice}!")
      self.trade(player, choice.capitalize())
    else:
      print("This place doesn't exist.")

  # Trade function to trade items for items
  def trade(self, player, place):

    print("Here's your inventory: ")
    player.show_inventory()

    # Dictionary of trades according to place
    trades = {

        "Blacksmith" : [
            {"item" : "Bread", "amt" : 10, "item2" : "Iron", "amt2" : 2},
            {"item" : "Iron", "amt" : 4, "item2" : "Pickaxe", "amt2" : 1},
            {"item" : "Diamond", "amt" : 1, "item2" : "Sword", "amt2" : 1},
        ],

        "Bakery": [
            {"item" : "Wheat", "amt" : 3, "item2" : "Bread", "amt2" : 2},
            {"item" : "Coal", "amt" : 3, "item2" : "Wheat Seeds", "amt2" : 5},
        ],

        "Grocery Store": [
            {"item" : "Gold", "amt" : 1, "item2" : "Carrot", "amt2" : 2},
            {"item" : "Gold", "amt" : 1, "item2" : "Mushrooms", "amt2" : 3},
            {"item" : "Gold", "amt" : 1, "item2" : "Meat", "amt2" : 2},
        ],

        "Plant Shop": [
            {"item" : "Mushrooms", "amt" : 6, "item2" : "Apple Seeds", "amt2" : 3},
            {"item" : "Berries", "amt" : 5, "item2" : "Mango Seeds", "amt2" : 2},
        ]
    }

    list_trades = trades.get(place)
    count = 1
    for trade in list_trades:
      print(f"{count}. Trade {trade['amt']} {trade['item']} for {trade['amt2']} {trade['item2']}")
      count += 1

    while True:
      try:
        choice = int(input("\nSelect a number to trade. 0 to exit."))
        if choice == 0:
          print("Leaving!")
          break

        if 1 <= choice <= len(list_trades):

          trade = list_trades[choice - 1]

          item_avail = False
          for item, quantity in player.inventory.items():
            if item == trade["item"] and quantity >= trade["amt"]:
              item_avail = True
              break

          if item_avail == True:
            player.remove(trade["item"], trade["amt"])
            player.obtain(trade["item2"], trade["amt2"])
            print("Trade successful!")
            break

          else:
            print("You don't have enough items to trade!")

        else:
          print("Invalid option.")
      except ValueError:
        print("Invalid input. Numbers only!")


# Garden Class

In [67]:
# Garden class organizes what is in the player's garden and all its available functions
class Garden:
  def __init__(self):

    # Garden starts empty
    self.garden = []

    # Default duration for plants to be harvestable is 10 seconds
    self.duration = 10

    # If whole garden had been updated since last planted seed
    self.water_update = False

  # View seeds function iterates through inventory to find only entries with "Seeds" in the name
  def view_seeds(self, player):

    seeds_list = []

    # Checks if inventory has seeds at all
    seeds_available = False

    print("\nHere are all your available seeds: ")

    # Loops through inventory, checking if "Seeds" is in name. If so, prints, and sets to True
    print("")
    for item, amount in player.inventory.items():
      if "Seeds" in item:
        seeds_list.append(item)
        print(f"{len(seeds_list)}. {item} : {amount}")
        seeds_available = True

    if seeds_available == False:
      print("No seeds available.")
      return False
    print("")

    return seeds_list

  # Plant function to take seed from inventory and add to garden
  def plant(self, player, seed):

    # If the seed is in inventory, you can plant
    if seed in player.inventory:

      # Plants have three attributes:
      # - Name of the seed
      # - Time of planting
      # - Watered or not
      plant = {
          "Seed Name" : seed,
          "Plant Time": time.time(),
          "Watered" : False
      }

      # Adds plant to garden
      self.garden.append(plant)
      print(f"{plant['Seed Name']} has been planted!")

      # New plant means whole garden hasn't been planted
      self.water_update = False

      # Removes from inventory
      player.remove(seed)

    else:
      print(f"You don't have anything to plant!")

  # Water function makes all available plants watered
  def water(self):

    # If garden has plants
    if self.garden:

      # Loops through garden
      for plant in self.garden:

        # If plant is not watered, sets it to true
        if plant["Watered"] == False:
          plant["Watered"] = True
          print(f"You watered {plant['Seed Name']}!")

      # Whole garden is now watered
      self.water_update = True
      print("\nYour whole garden is watered!")

    else:
      print("There's no plants in your garden!")

  # View garden function
  def view_garden(self):

    # Only if there are plants in garden
    if self.garden:
      print("\nGarden: ")

      # Loops through every plant in garden
      for plant in self.garden:

        # Checks if plant is ready to be harvested (grown and watered)
        if self.check_growth(plant) == True and plant["Watered"] == True:
          plant_status = "Ready for harvest!"

        # Still growing if both aren't true
        else:
          plant_status = "Still growing!"

        # If watered is true or false
        if plant["Watered"] == True:
          watered_or_not = "Watered"
        else:
          watered_or_not = "Not Watered"

        # Print out the attributes
        print("")
        print(f"Plant: {plant['Seed Name']}")
        print(f"- Status: {plant_status}")
        print(f"- Watered or Not: {watered_or_not}\n")
        print("")

    # If garden has no plants
    else:
      print("Your garden is empty!")

  # Harvest function checks if plant is watered and grown.
  # If so, then obtain 3 of the plant's yield and delete from garden
  def harvest(self):

    # Loop through every plant in garden
    if self.garden == []:
      print("Your garden is empty!")
    else:
      harvestable = False
      for plant in self.garden:

        # If watered and grown
        if plant["Watered"] and self.check_growth(plant) == True:

          # Add 3 of yield to inventory
          plant_yield = plant["Seed Name"].replace(" Seeds", "")
          player.obtain(plant_yield, 3)
          print(f"You've harvested 3 {plant_yield}!")

          # Remove from garden
          self.garden.remove(plant)

          harvestable = True
      if harvestable == False:
        print("No plants are ready for harvest!")


  # Checks if difference of current time and time of planting is past duration
  def check_growth(self, plant):
    return (time.time() - plant["Plant Time"]) >= self.duration

  # Stay in garden loop for actions
  def garden_loop(self, player):
    while True:
      print("")
      print("""You're in your garden! Actions:
      1. Plant
      2. Water
      3. Harvest
      4. View Garden
      5. View Inventory
      6. Leave """)
      action = int(input("\nSelect a number. "))

      if action == 1:

        seeds_list = self.view_seeds(player)

        if seeds_list == False:
          print("You have no seeds to plant.")

        else:
          while True:
            try:
              choice = int(input("What would you like to plant? "))
              if 1 <= choice <= len(seeds_list):
                self.plant(player, seeds_list[choice - 1])
                break
              else:
                print("Invalid number.")
            except ValueError:
              print("Numbers only!")

      elif action == 2:
        self.water()

      elif action == 3:
        self.harvest()

      elif action == 4:
        self.view_garden()

      elif action == 5:
        player.show_inventory()

      elif action == 6:
        print("Leaving garden!")
        return False

      else:
        print("Invalid input.")

# Game Class

In [69]:
class Game:
  def __init__(self, player):
    self.player = player
    self.biomes = [Forest(), Meadow(), Caves(), Town()]
    self.location = None

  # Travel function to go to biomes
  def travel(self):
    print("\nAvailable biomes:")
    for biome in self.biomes:
      print(f"{biome.name}")
    print("")

    while True:
      choice = input("Which biome would you like to visit? ")
      for biome in self.biomes:
        if biome.name == choice.capitalize() and biome.unlocked == True:
          break
      else: # https://www.w3schools.com/python/gloss_python_for_else.asp
        print("Invalid biome")
        continue
      break

    for biome in self.biomes:

      if biome.name == choice.capitalize() and biome.unlocked == False:
        while True:
          choice = input(f"{biome.name} is still locked. \nType 'yes' to attempt to unlock it. Type 'no' to quit.")
          if choice == "yes":
            biome.unlock(player)
            self.biome_action()
            return False
          elif choice == "no":
            print("Returning to main page")
            return False
          else:
            print("Invalid input. Type 'yes' or 'no'.")
            return True

      elif biome.name == choice.capitalize() and biome.unlocked == True:
        print(f"\nYou are now in {biome.name}")
        self.location = biome
        self.biome_action()

  # Biome specific actions
  def biome_action(self):
    while self.location:
      print("\nWhat would you like to do? You can explore, show inventory, consume, show pets, or return.")
      action = input("I'd like to... ")

      if action == "explore":
        print("")
        self.location.explore(self.player)

      elif action == "show inventory":
        player.show_inventory()

      elif action == "consume":
        player.consume()

      elif action == "show pets":
        player.show_pets()

      elif action == "return":
        self.location = None
        break

      else:
        print("Invalid action.")

    return True

  # Main page actions
  def start_action(self):
    print("\nChoose an action: travel, consume, show inventory, show pets, quit")
    action = input("I'd like to... ").lower()

    if action == "travel":
      self.travel()

    elif action == "consume":
      player.consume()

    elif action == "show inventory":
      player.show_inventory()

    elif action == "show pets":
      player.show_pets()

    elif action == "quit":
      return False

    else:
      print("Invalid action.")

    return True

  # Start the loop
  def start(self):

    print("""Rules:
    You can only access Caves with a pickaxe in your inventory.
    You can only relax in your House.""")

    while True:
      if self.start_action() == False:
        break

# Game Loop

In [71]:
username = input("Enter your username: ")
player = Player(username)
game = Game(player)
garden = Garden()

# Couldn't figure out how to make biome rules work sadly :(

game.start()

Enter your username:  p



Rules:
    You can only access Caves with a pickaxe in your inventory.
    You can only relax in your House.

Choose an action: travel, consume, show inventory, show pets, quit


I'd like to...  travel



Available biomes:
Forest
Meadow
Caves
Town



Which biome would you like to visit?  forest


...

I'd like to...  show inventory



Inventory:
Mango Seeds: 1
Wheat Seeds: 1
Apple Seeds: 1
Meat: 2
Bread: 2
Iron: 3
Diamond: 1


What would you like to do? You can explore, show inventory, consume, show pets, or return.


I'd like to...  return



Choose an action: travel, consume, show inventory, show pets, quit


I'd like to...  quit
